In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pds
import openpyxl
#importing the libraries
file = r'C:\Users\Super\Downloads\Cohesive Zone(1)(1).xlsx'#accessing the exel file with the temperature differential data
data = pd.ExcelFile(file)
print(data.sheet_names) #this returns all the sheets in the excel file
df = data.parse('Sheet1')# accessing sheet1(a particular sheet within the workbook)
df.info
ps = openpyxl.load_workbook(r'C:\Users\Super\Downloads\Cohesive Zone(1)(1).xlsx')#working in the sheet
sheet = ps['Sheet1'] 
stave_position_final=list() #declaring lists that will contain the 
stave_angle_final=list()
stave_temp_diff_final=list()
for ultra in range(0,26):  # because we are taking stave temperature differential standard deviations for every hour and for 24 hours and 6 hours(total 24+2)
    for count in range(ultra+5,ultra+6): #accesing excel row (in my case info started from row 5 of excel)
        stave_angle=list()
        for i in range(1,49):
            stave_angle.append(7.5*i) #initiating stave angle list with uniform spacings of 7.5 degree at a particular height along the circumferential plane 
        stave_angle_final.append(stave_angle)
        stave_position1=['ROW6 (19.8m)','ROW7 (21.07m)','ROW8 (23.6m)','ROW9 (25.4m)'] #declaring stave positions
        stave_position_final.append(stave_position1)
        list1=['B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y'] #accessing row 6 info coloumwise from excel sheet
        stave_temp1_diff=list() # declaring initial row 6 temp reading list for interpolation

        for j in list1:
            stave_temp1_diff.append(sheet[str(j)+str(count)].value) #appending values from excel sheet for list1
        stave_temp_interpolate=list() #declaring interploation list
        for i in range(0,len(stave_temp1_diff)-1):
            stave_temp_interpolate.append((stave_temp1_diff[i]+stave_temp1_diff[i+1])/2) #interpolation step..average of two successive readings in loop
        stave_temp_interpolate.append((stave_temp1_diff[0]+stave_temp1_diff[23])/2) # only 23 readings will be obtained from previous loop...so generating one more reading taking average of 1st and 23rd reading
        
        print(len(stave_temp_interpolate)) #printing length for verification
        stave_temp_diff=list() 
        for i in range(0,24):
            stave_temp_diff.append(stave_temp_interpolate[i]) #appending new interpolated data into a new list stave_temp_diff
            stave_temp_diff.append(stave_temp1_diff[i]) #appending original 24 readings
        print(len(stave_temp_diff)) #verification by printing of length
        string1='Z AA AB AC AD AE AF AG AH AI AJ AK AL AM AN AO AP AQ AR AS AT AU AV AW AX AY AZ BA BB BC BD BE BF BG BH BI BJ BK BL BM BN BO BP BQ BR BS BT BU BV BW BX BY BZ CA CB CC CD CE CF CG CH CI CJ CK CL CM CN CO CP CQ CR CS CT CU CV CW CX CY CZ DA DB DC DD DE DF DG DH DI DJ DK DL DM DN DO DP DQ DR DS DT DU DV DW DX DY DZ EA EB EC ED EE EF EG EH EI EJ EK EL EM EN EO EP EQ ER ES ET EU EV EW EX EY EZ FA FB FC FD FE FF FG FH FI FJ FK FL FM'
        # in the above case..I have generated a list containing coloumns of temperature readings of Row7, Row8 and Row9....inefficient but does the job
        list2=string1.split()

        for j in list2:
            stave_temp_diff.append(sheet[str(j)+str(count)].value) #appending all the temp differential data of row 7,8 and 9 into the list
    
        stave_temp_array=np.array(stave_temp_diff) #converting list into array
        stave_temp_arr=stave_temp_array.reshape(4, 48) #reshaping for dimensions to match for subsequent contour plotting
        stave_temp_diff_final.append(stave_temp_arr) #appending this array into a list that will then be looped to create contour plots corresponding to list index readings


In [ ]:
# NOW WE OBTAIN CONTOUR PLOTS AND SAVE THOSE IMAGES
import os 
os.chdir('DEVDATA2') #we create a folder to store saved images..and change directory to that folder

for ultra in range(0,24):# for hourly readings...ie one hour standard deviation readings
            plt.title('Std deviation of 1 Hour temperature'+" "+"("+str(ultra)+"-"+str(ultra+1)+")"+"hrs") #title
            c=plt.contourf(stave_angle_final[ultra], stave_position_final[ultra], stave_temp_diff_final[ultra], 30, cmap='RdGy') #contour plot
            plt.xlabel('ANGLE IN DEGREES') #x axis labeling
            plt.ylabel('ROW NUMBER') #y axis labeling
            
            t=plt.colorbar(c); #actual generation of plot
            t.set_label('Time Differential of stave temperature (\N{DEGREE SIGN}C/min)',rotation=270,fontsize=11,labelpad=17) #setting label for the colorbar that indicates stave temp differential
            plt.savefig(str(ultra)+".png",bbox_inches="tight") #saving the image in png format in the folder...note save as just 'number'.png...not as any character.png 
            plt.clf() #clearing the image...if this is not mentioned we will obtain superposition of different contour plots over same original plot...hence clear once saved

In [ ]:
pip install opencv-python #installing opencv package for subsequent gif creation

In [ ]:
import cv2 # importing required libraries
import os
import imageio 
path_parent = os.path.dirname(os.getcwd()) #we are in saved image directory..this step gets us back to our working directory..or main working frame
os.chdir(path_parent) #change directory to present working directory
def generate_gif(folder): # generate a function
    image_names = [] #image names will create an array where we will subsequently append our stored images
    for filename in os.listdir(folder):#accesses our folder containing saved images
        name=os.path.join(folder,filename)
        image = cv2.imread(os.path.join(folder,filename)) #accesses  the images
        if image is not None:
            image_names.append(name) #appends images in array image_names
    image_names_sorted = sorted(image_names,key=lambda x: int(x.split('\\')[1].split('.')[0])) #sorts images based on numbering...for chronological appearence in gif
    images=[] #declaring array images
    for i in range(len(image_names_sorted)):
        images.append(imageio.imread(image_names_sorted[i])) #appending sorted images
    imageio.mimwrite('result.gif', images,'GIF',fps=0.5) #main gif generator function..speed can be controlled with fps
    